# A fresh start

In [17]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye.oechem import *
from openeye.oeomega import *
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer
from networkx.algorithms import isomorphism

## Step 1

Generate beta-cyclodextrin and benzoate[-1] from SMILES strings then grab the charges for these molecules from existing `mol2` files containing AM1-BCC charges.

In [2]:
bcd_smiles = 'C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O'
# https://pubchem.ncbi.nlm.nih.gov/compound/517055#section=Top
ben_smiles = '[O-]C(=O)c1ccccc1'

smiles = [bcd_smiles, ben_smiles]
names = ['bcd', 'ben']
molecules = [] * len(smiles)

In [3]:
for (smile, name) in zip(smiles, names):
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smile)
    print ("Canonical isomeric SMILES is", OEMolToSmiles(mol))
    oechem.OEAddExplicitHydrogens(mol)
    mol.SetTitle(name)
    
#     omegaOpts = OEOmegaOptions()
#     omegaOpts.SetMaxConfs(1)
#     omega = OEOmega(omegaOpts)
#     omega(mol)
    
    molecules.append(mol)

Canonical isomeric SMILES is C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O
Canonical isomeric SMILES is c1ccc(cc1)C(=O)[O-]


Instead of using `omega` to generate the structure, which takes a long time or chokes on bcd, I'll grab the coordinates from the `mol2` file.

## Step 2
Make the mapping using **link**...

In [6]:
topologies = [] * len(molecules)
graphs = [] * len(molecules)

for mol in molecules:
    topology = generateTopologyFromOEMol(mol)
    topologies.append(topology)
    
    graph = generateGraphFromTopology(topology)
    graphs.append(graph)

In [14]:
path = 'bcd-ben/'
reference_files = ['bcd.mol2', 'ben.mol2']
names = ['bcd-reference',  'ben-reference']

for (reference, name) in zip(reference_files, names):
    mol = oechem.OEMol()
    ifs = oechem.oemolistream(path + reference)
    for mol in ifs.GetOEGraphMols():
        # Make sure we have unique atom names for the mapping, or else things will be sticky.
        oechem.OETriposAtomNames(mol)
        print ("Canonical isomeric SMILES is", OEMolToSmiles(mol))
        mol.SetTitle(name)
        reference_molecules.append(mol)

Canonical isomeric SMILES is C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H]([Os]1)[Os][C@@H]3[C@H]([Os][C@@H]([C@@H]([C@H]3O[Ho])O[Ho])[Os][C@@H]4[C@H]([Os][C@@H]([C@@H]([C@H]4O[Ho])O[Ho])[Os][C@@H]5[C@H]([Os][C@@H]([C@@H]([C@H]5O[Ho])O[Ho])[Os][C@@H]6[C@H]([Os][C@@H]([C@@H]([C@H]6O[Ho])O[Ho])[Os][C@@H]7[C@H]([Os][C@@H]([C@@H]([C@H]7O[Ho])O[Ho])[Os][C@@H]8[C@H]([Os][C@H]([Os]2)[C@@H]([C@H]8O[Ho])O[Ho])CO[Ho])CO[Ho])CO[Ho])CO[Ho])CO[Ho])CO[Ho])O[Ho])O[Ho])O[Ho]
Canonical isomeric SMILES is [C+]([O-])([O-])[Ca]1[CaH][CaH][CaH][CaH][CaH]1


In [40]:
reference_topologies = [] * len(reference_molecules)
reference_graphs = [] * len(reference_molecules)

for mol in reference_molecules:
    topology = generateTopologyFromOEMol(mol)
    reference_topologies.append(topology)
    
    graph = generateGraphFromTopology(topology)
    reference_graphs.append(graph)

In [43]:
print(reference_graphs[0].size())

0


In [38]:
reference_to_topology_atom_mappings = [] * len(molecules)

for (reference, target) in zip(reference_graphs, graphs):
    print(reference.size())
    print(target.size())

    GM = isomorphism.GraphMatcher(target, reference)
    if GM.is_isomorphic():
        reference_to_topology_atom_mapping = dict()
        for (topology_atom, reference_atom) in GM.mapping.items():
            reference_to_topology_atom_mapping[reference_atom] = topology_atom
        reference_to_topology_atom_mappings.append(reference_to_topology_atom_mapping)

0
154
0
14


In [35]:
gm = isomorphism.GraphMatcher(reference_graphs[0], graphs[0])

In [30]:
gm.is_isomorphic()

False

In [31]:
gm.mapping

{}

## Step 3: the complex